In [1]:
%matplotlib inline
import math
import pandas as pd
from tsmom_model3 import *
import numpy as np
from datetime import datetime
import linearmodels as lm
import statsmodels.api as sm

/Users/mrefermat/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
import pylab
sns.set(font_scale=2)
pylab.rcParams['figure.figsize'] = (24, 18)

# Hand cleansed
Hand cleaned to removed duplicates

In [3]:
cleansed=load_amihud_markets_price()

/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model3.py:36: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  data=pd.read_csv('./Price.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='last')
/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model3.py:38: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  qd=pd.read_csv('./liquid_contracts.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='last')


# Amihud methodology

Amihud (2002) developed the illiquidity measure expressed by:

$$ Amihud =\frac{1}{N} \sum \frac{|r_t|}{\text{dvol}_t} $$

where $r_t$ is the return on day $t$ and $Volume_t$ is the dollar volume on day $t$. 

The benefit of taking the daily return is to weight the move by change of price and the volume.


In [4]:
amihud=calculate_amihud_liquidity(cleansed)[:'2016'] 

/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model3.py:25: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).mean()
  data=pd.read_csv('./Volume.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='mean')
/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model3.py:27: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).sum()
  v=pd.read_csv('./volume_data.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='sum')
/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model3.py:29: FutureWarning: pd.rolling_mean is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(window=250,min_periods=100,center=False).mean()
  volume=pd.rolling_mean(v,250,min_periods=100).resample(rule='d',how='mean')[:'2016']
/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model3.py:29: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...)

In [5]:
sector_resid={}
sector_rsq={}
d_map=sector_map(amihud)
for sect in d_map.keys():
    sector_data={}
    rsq=pd.Series()
    for mkt in d_map[sect]:
        ind2=amihud[mkt].dropna().index
        ind=amihud[mkt].shift().loc[ind2].dropna().index
        res=sm.OLS(amihud[mkt].shift().loc[ind],amihud[mkt].loc[ind]).fit()
        sector_data[mkt]=res.resid
        rsq[mkt]=res.rsquared
    sector_resid[sect]=pd.DataFrame(sector_data)
    sector_rsq[sect]=rsq.mean()


In [6]:
sector_rtns={}
d_map=sector_map(amihud)
for sect in d_map.keys():
    sector_rtns[sect]=cleansed[d_map[sect]].dropna(how='all')

In [34]:
results_df=pd.DataFrame()
for sect in sector_resid.keys():
    c_matrix=[]
    tstats=[]
    for t in [0,1]:
        for ent in [0,1]:
            data=pd.DataFrame(sector_resid[sect].stack()) 
            px=pd.DataFrame(sector_rtns[sect].resample(rule='m',how='last').pct_change().shift().stack())
            ind=data.index
            px=px.loc[ind]
            res=lm.PanelOLS(px.to_panel()[0],
                    data.to_panel()[0], 
                    time_effects=t,
                    entity_effects=ent).fit(cov_type='clustered',cluster_time=True)
            c_matrix.append(res.params.Exog)
            tstats.append(res.tstats.Exog)
    results_df[sect]=pd.Series(c_matrix)/10000.0
    results_df[sect + 'Tstats']=pd.Series(tstats)
results_df=results_df.set_index([['None','Ent','Time','Ent and Time']])

/Users/mrefermat/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  
/Users/mrefermat/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.

  # This is added back by InteractiveShellApp.init_path()
/Users/mrefermat/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively

In [35]:
results_df.T.round(2)

,None,Ent,Time,Ent and Time
Equities,0.23,0.20,0.11,0.10
EquitiesTstats,2.01,1.66,2.96,2.73
Commodities,-1.32,-1.80,0.01,0.13
CommoditiesTstats,-0.27,-0.36,0.00,0.03
Currencies,-0.24,-0.24,-0.25,-0.25
CurrenciesTstats,-1.33,-1.31,-1.37,-1.19
Fixed Income,-1078.27,-1101.20,-541.79,-434.23
Fixed IncomeTstats,-1.35,-1.35,-0.52,-0.40


# FHT method

In [9]:
FHT=calculate_FHT(cleansed)

/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model3.py:53: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).count()
  nonzero=x[x==0].resample(rule='m',how='count')
/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model3.py:54: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).count()
  Z=(nonzero/x.resample(rule='m',how='count')).dropna()
/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model3.py:55: FutureWarning: pd.rolling_std is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=12,center=False).std()
  data[c]=pd.Series(norm.cdf((1+Z)/2.),index=Z.index)*2*pd.rolling_std(x,12).resample(rule='m',how='last')
/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model3.py:55: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  data[c]=pd.Series(norm.cdf((1+Z)/2.),index=Z.index)*2*pd.rolling_std(x,12).resample(

In [10]:
sector_resid_FHT={}
sector_rsq_FHT={}
d_map=sector_map(amihud)
for sect in d_map.keys():
    sector_data={}
    rsq=pd.Series()
    for mkt in d_map[sect]:
        ind2=FHT[mkt].dropna().index
        ind=FHT[mkt].shift().loc[ind2].dropna().index
        res=sm.OLS(FHT[mkt].shift().loc[ind],FHT[mkt].loc[ind]).fit()
        sector_data[mkt]=res.resid
        rsq[mkt]=res.rsquared
    sector_resid_FHT[sect]=pd.DataFrame(sector_data)
    sector_rsq_FHT[sect]=rsq.mean()

In [11]:
results_df_FHT=pd.DataFrame()
for sect in sector_resid_FHT.keys():
    c_matrix=[]
    tstats=[]
    for t in [0,1]:
        for ent in [0,1]:
            data=pd.DataFrame(sector_resid_FHT[sect].stack()) 
            px=pd.DataFrame(sector_rtns[sect].resample(rule='m',how='last').pct_change().shift().stack())
            ind=data.index
            px=px.loc[ind]
            res=lm.PanelOLS(px.to_panel()[0],
                    data.to_panel()[0], 
                    time_effects=t,
                    entity_effects=ent).fit(cov_type='clustered',cluster_time=True)
            c_matrix.append(res.params.Exog)
            tstats.append(res.tstats.Exog)
    results_df_FHT[sect]=pd.Series(c_matrix)*100
    results_df_FHT[sect + 'Tstats']=pd.Series(tstats)
results_df_FHT=results_df_FHT.set_index([['None','Ent','Time','Ent and Time']])

/Users/mrefermat/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  
/Users/mrefermat/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.

  # This is added back by InteractiveShellApp.init_path()
/Users/mrefermat/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively

In [12]:
results_df_FHT.T.round(2)

,None,Ent,Time,Ent and Time
Equities,44.30,33.83,74.70,71.50
EquitiesTstats,1.14,0.85,1.74,1.62
Commodities,-0.42,-0.50,-0.14,-0.17
CommoditiesTstats,-1.62,-2.35,-0.59,-0.73
Currencies,50.91,61.17,53.04,63.26
CurrenciesTstats,0.46,0.50,0.47,0.44
Fixed Income,-25.16,-36.86,-34.87,-40.38
Fixed IncomeTstats,-0.60,-0.84,-0.84,-0.92


# Presentation
First rsquares of residuals

In [36]:
print(results_df.T.round(2).to_latex())

\begin{tabular}{lrrrr}
\toprule
{} &     None &      Ent &    Time &  Ent and Time \\
\midrule
Equities           &     0.23 &     0.20 &    0.11 &          0.10 \\
EquitiesTstats     &     2.01 &     1.66 &    2.96 &          2.73 \\
Commodities        &    -1.32 &    -1.80 &    0.01 &          0.13 \\
CommoditiesTstats  &    -0.27 &    -0.36 &    0.00 &          0.03 \\
Currencies         &    -0.24 &    -0.24 &   -0.25 &         -0.25 \\
CurrenciesTstats   &    -1.33 &    -1.31 &   -1.37 &         -1.19 \\
Fixed Income       & -1078.27 & -1101.20 & -541.79 &       -434.23 \\
Fixed IncomeTstats &    -1.35 &    -1.35 &   -0.52 &         -0.40 \\
\bottomrule
\end{tabular}



In [13]:
print(results_df_FHT.T.round(2).to_latex())

\begin{tabular}{lrrrr}
\toprule
{} &   None &    Ent &   Time &  Ent and Time \\
\midrule
Equities           &  44.30 &  33.83 &  74.70 &         71.50 \\
EquitiesTstats     &   1.14 &   0.85 &   1.74 &          1.62 \\
Commodities        &  -0.42 &  -0.50 &  -0.14 &         -0.17 \\
CommoditiesTstats  &  -1.62 &  -2.35 &  -0.59 &         -0.73 \\
Currencies         &  50.91 &  61.17 &  53.04 &         63.26 \\
CurrenciesTstats   &   0.46 &   0.50 &   0.47 &          0.44 \\
Fixed Income       & -25.16 & -36.86 & -34.87 &        -40.38 \\
Fixed IncomeTstats &  -0.60 &  -0.84 &  -0.84 &         -0.92 \\
\bottomrule
\end{tabular}

